In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import random

from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

import seaborn as sns
from scipy.sparse.linalg import eigs

from tensorflow.keras.utils import to_categorical

In [ ]:
path = '/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/'
os.listdir(path)

In [ ]:
data = pd.read_csv(path+'icml_face_data.csv')
data.head(5)

In [ ]:
data[' Usage'].unique()

In [ ]:
def prepare_data(data):
    """ Prepare data for modeling 
        input: data frame with labels und pixel data
        output: image and label array """
    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label


In [ ]:
emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

train_image_array, train_image_label = prepare_data(data[data[' Usage']=='Training'])
val_image_array, val_image_label = prepare_data(data[data[' Usage']=='PrivateTest'])
test_image_array, test_image_label = prepare_data(data[data[' Usage']=='PublicTest'])

In [ ]:
train_images = train_image_array.reshape((train_image_array.shape[0], 1, 48, 48))
train_images = train_images.astype('float32')/255
val_images = val_image_array.reshape((val_image_array.shape[0], 1, 48, 48))
val_images = val_images.astype('float32')/255
test_images = test_image_array.reshape((test_image_array.shape[0], 1, 48, 48))
test_images = test_images.astype('float32')/255

train_labels = to_categorical(train_image_label)
val_labels = to_categorical(val_image_label)
test_labels = to_categorical(test_image_label)

In [ ]:
train_labels = to_categorical(train_image_label)
val_labels = to_categorical(val_image_label)
test_labels = to_categorical(test_image_label)

In [ ]:
data[' Usage'].value_counts()

In [ ]:
((data[data[' Usage']=='Training']['emotion'].value_counts()).sort_index())

In [ ]:
# Calculating the percentage into each category in the training dataset
percentage = (((data[data[' Usage']=='Training']['emotion'].value_counts()).sort_index()) * 100 /len(data[data[' Usage']=='Training']['emotion']))

class_weight = dict(zip(range(0, 7), percentage.tolist()))
class_weight

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(20,5))
sns.countplot(data = data[data[' Usage']=='Training'], x='emotion', ax=ax1).set_title('Training')
ax1.set_xticklabels(emotions.values())
sns.countplot(data = data[data[' Usage']=='PublicTest'], x='emotion', ax=ax2).set_title('Testing')
ax2.set_xticklabels(emotions.values())
sns.countplot(data = data[data[' Usage']=='PrivateTest'], x='emotion', ax=ax3).set_title('Validation')
ax3.set_xticklabels(emotions.values())

In [ ]:
# Randomly plotting 20 images from the dataset

def sample_plot(x,y=None):
    #x, y are numpy arrays
    n = 20
    samples = random.sample(range(x.shape[0]),n)
    
    fig, axs = plt.subplots(2,10, figsize=(25,5), sharex=True, sharey=True)
    ax = axs.ravel()
    for i in range(n):
        ax[i].imshow(x[samples[i],:,:], cmap=plt.get_cmap('gray'))
        ax[i].set_xticks([])
        ax[i].set_yticks([])
        if y is not None:
            ax[i].set_title(emotions[y[samples[i]]])
            
sample_plot(train_image_array, train_image_label)

In [ ]:
def make_image_vector(image_list, new_size):
    D = np.empty([len(image_list), new_size])
    i = 0
    for image in image_list:
        D[i, :] = image
        i += 1
    return D

def avgfaces_by_emotion(df):
    
    avgfaces = []
    for emotion in sorted(df['emotion'].unique()):
        sub_df = df[df['emotion']==emotion]    
        avg_pixels = np.mean(sub_df['pixels'].values)
        avgfaces.append((avg_pixels, emotion))
        
    return avgfaces

def eigenfaces_by_emotion(df, k=1):
    
    variance_arr = []
    eigenfaces = []
    for emotion in sorted(df['emotion'].unique()):
        sub_df = df[df['emotion'] == emotion]
        
        pixels = list(sub_df['pixels'].values)
        D = make_image_vector(pixels, 48*48)
        A = np.dot(D.T, D)
        
        # Find k largest magnitude (LM) eigenvectors
        vals, vecs = eigs(A, k, which = 'LM')
        variance = vals
        
        eigenfaces.extend([(vec.reshape(48, 48).astype(float), emotion) for vec in vecs.T])
        variance_arr.extend([variance])
        
    return variance_arr, eigenfaces

In [ ]:
df = data.copy()
df['pixels'] = df[' pixels'].apply(lambda x: np.array(x.split(), dtype=np.float))
avgfaces = avgfaces_by_emotion(df)

var, eigenfaces = eigenfaces_by_emotion(df, 1)

In [ ]:
# Plot faces and show emotion label
def plot_face_and_emotion(plot_nrow, plot_ncol, face_list):
    fig, axes = plt.subplots(plot_nrow, plot_ncol, figsize=(15, 15))
    for i, (face, emotion) in enumerate(face_list):
        ax = axes.ravel()[i]
        ax.imshow(face.reshape(48, 48), cmap='gray')
        # Turn off tick labels
        ax.set_xlabel(emotions[emotion])
        ax.set_yticklabels([])
        ax.set_xticklabels([])

In [ ]:
avg = []
avg_pixels = np.mean(df['pixels'].values)
avg.append((avg_pixels, '1'))

fig, axes = plt.subplots(1, 1, figsize=(5, 5))
for i, (face, emotion) in enumerate(avg):
    ax = axes
    ax.imshow(face.reshape(48, 48), cmap='gray')
    # Turn off tick labels

In [ ]:
plot_face_and_emotion(1, 7, avgfaces)

In [ ]:
plot_face_and_emotion(1, 7, eigenfaces)

# CNN

In [ ]:
import tensorflow as tf
from keras import models
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import TopKCategoricalAccuracy

In [ ]:
train_images = train_image_array.reshape((train_image_array.shape[0], 48, 48, 1))
train_images = train_images.astype('float32')/255
val_images = val_image_array.reshape((val_image_array.shape[0], 48, 48, 1))
val_images = val_images.astype('float32')/255
test_images = test_image_array.reshape((test_image_array.shape[0], 48, 48, 1))
test_images = test_images.astype('float32')/255

train_labels = to_categorical(train_image_label)
val_labels = to_categorical(val_image_label)
test_labels = to_categorical(test_image_label)

In [ ]:
cnn_model =  models.Sequential([
                        Conv2D(128, (3,3), activation = 'relu', strides = 1, padding = 'same', input_shape = (48, 48, 1)),
                        MaxPool2D((2,2), strides = 2 , padding = 'same'),
                        Conv2D(128, (3,3), activation = 'relu', strides = 1, padding = 'same'),
                        MaxPool2D((2,2), strides = 2 , padding = 'same'),
                               
                        Conv2D(64, (3,3), activation = 'relu', strides = 1, padding = 'same'),
                        MaxPool2D((2,2), strides = 2 , padding = 'same'),
                        Conv2D(64, (3,3), activation = 'relu', strides = 1, padding = 'same'),
                        MaxPool2D((2,2), strides = 2 , padding = 'same'),
    
                        Conv2D(32, (3,3), activation = 'relu', strides = 1, padding = 'same'),
                        MaxPool2D((2,2), strides = 2 , padding = 'same'),
                        Conv2D(32, (3,3), activation = 'relu', strides = 1, padding = 'same'),
                        MaxPool2D((2,2), strides = 2 , padding = 'same'),
                        
                        Flatten(),
                        Dense(128, 'relu'),
                        Dense(7,'softmax'),
                ])

In [ ]:
cnn_model.compile(optimizer = Adam(lr=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3)])
cnn_model.summary()

In [ ]:
epochs = 5
trained_model_conv = cnn_model.fit(train_images, 
                               train_labels, 
                               epochs = epochs, 
                               batch_size = 32, 
                               class_weight = class_weight,
                               validation_data = (val_images, val_labels))

In [ ]:
plt.plot(trained_model_conv.history['accuracy'], label = 'Train Accuracy')
plt.plot(trained_model_conv.history['val_accuracy'], label = 'Val Accuracy')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(trained_model_conv.history['top_k_categorical_accuracy'], label = 'Train Accuracy Top 3')
plt.plot(trained_model_conv.history['val_top_k_categorical_accuracy'], label = 'Val Accuracy Top 3')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
test_loss, test_acc, top_k_acc = cnn_model.evaluate(test_images, test_labels)
print('test accuracy :', test_acc)
print('test accuracy top 3:', top_k_acc)

## Image Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center = False,  # set input mean to 0 over the dataset
        samplewise_center = False,  # set each sample mean to 0
        featurewise_std_normalization = False,  # divide inputs by std of the dataset
        samplewise_std_normalization = False,  # divide each input by its std
        zca_whitening = False,  # apply ZCA whitening
        rotation_range = 10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range = 0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range = 0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = False,  # randomly flip images
        vertical_flip = False)  # randomly flip images

In [ ]:
# Data augment over all data
datagen.fit(train_images)

for X_batch, y_batch in datagen.flow(train_images, train_labels, batch_size = 9):
    fig, axes = plt.subplots(3, 3, figsize = (8, 8))
    for i in range(0, 9):
        ax = axes.ravel()[i]
        ax.imshow(X_batch[i].reshape(48, 48, 1))
        ax.set_xlabel(y_batch[i])
    break

emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

In [ ]:
aug_model = cnn_model.fit(
                 datagen.flow(train_images, train_labels, batch_size = 32),
                 validation_data = (val_images, val_labels),
                 steps_per_epoch = len(train_images) / 32, 
                 epochs = 5)

In [ ]:
test_loss, test_acc, top_k_acc = aug_model.evaluate(test_images, test_labels)
print('test accuracy :', test_acc)
print('test accuracy top 3:', top_k_acc)

In [ ]:
plt.plot(aug_model.history['accuracy'], label = 'Train Accuracy')
plt.plot(aug_model.history['val_accuracy'], label = 'Val Accuracy')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(aug_model.history['top_k_categorical_accuracy'], label = 'Train Accuracy Top 3')
plt.plot(aug_model.history['val_top_k_categorical_accuracy'], label = 'Val Accuracy Top 3')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# Augment just disgust images

indices = np.argwhere(train_image_label == 1)
indices = indices.reshape(indices.shape[0],)

datagen.fit(train_images[indices])

for X_batch, y_batch in datagen.flow(train_images[indices], train_labels[indices], batch_size = 9):
    fig, axes = plt.subplots(3, 3, figsize = (8, 8))
    for i in range(0, 9):
        ax = axes.ravel()[i]
        ax.imshow(X_batch[i].reshape(48, 48, 1))
        ax.set_xlabel(y_batch[i])
    break


In [ ]:
cnn_model2 =  models.Sequential([
                        Conv2D(128, (3,3), activation = 'relu', strides = 1, padding = 'same', input_shape = (48, 48, 1)),
                        MaxPool2D((2,2), strides = 2 , padding = 'same'),
                        Conv2D(128, (3,3), activation = 'relu', strides = 1, padding = 'same'),
                        MaxPool2D((2,2), strides = 2 , padding = 'same'),
                               
                        Conv2D(64, (3,3), activation = 'relu', strides = 1, padding = 'same'),
                        MaxPool2D((2,2), strides = 2 , padding = 'same'),
                        Conv2D(64, (3,3), activation = 'relu', strides = 1, padding = 'same'),
                        MaxPool2D((2,2), strides = 2 , padding = 'same'),
    
                        Conv2D(32, (3,3), activation = 'relu', strides = 1, padding = 'same'),
                        MaxPool2D((2,2), strides = 2 , padding = 'same'),
                        Conv2D(32, (3,3), activation = 'relu', strides = 1, padding = 'same'),
                        MaxPool2D((2,2), strides = 2 , padding = 'same'),
                        
                        Flatten(),
                        Dense(128, 'relu'),
                        Dense(7,'softmax'),
                ])

cnn_model2.compile(optimizer = Adam(lr=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3)])

In [ ]:
epochs = 5
for e in range(epochs):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in datagen.flow(train_images[indices], train_labels[indices], batch_size = 128):
        
        train_images_new = np.concatenate((train_images, x_batch), axis = 0)
        train_labels_new = np.concatenate((train_labels, y_batch), axis = 0)
        
        cnn_model2.fit(train_images_new, train_labels_new, validation_data = (val_images, val_labels))
        batches += 1
        
        if batches >= 2:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break


In [ ]:
test_loss, test_acc, top_k_acc = cnn_model2.evaluate(test_images, test_labels)
print('test accuracy :', test_acc)
print('test accuracy top 3:', top_k_acc)